In [1]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
data = pd.read_csv(
    "assignment2_data/assignment_2_train.csv"
)
print("train.shape = {} rows, {} cols".format(*data.shape))
data.head(n=2)

train.shape = 180000 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data_lb = pd.read_csv(
    "assignment2_data/assignment_2_test.csv"
)
print("train.shape = {} rows, {} cols".format(*data_lb.shape))
data_lb.head(n=2)

train.shape = 100001 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.0,W,12473,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.0,W,15651,417.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Основное задание:

Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.
​
Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

### Задание 1: 
сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [4]:
data['TransactionID'].unique

<bound method Series.unique of 0         2987000
1         2987001
2         2987002
3         2987003
4         2987004
           ...   
179995    3166995
179996    3166996
179997    3166997
179998    3166998
179999    3166999
Name: TransactionID, Length: 180000, dtype: int64>

In [5]:
def recat(series, vals):
    return series.map({v: i for i, v in enumerate(vals)})

cat_cols = [col for col, val in (data.dtypes == 'object').items() if val]
for col in cat_cols:
    vals = data[col].unique()
    
    data[col] = recat(data[col], vals)
    data_lb[col] = recat(data_lb[col], vals)

In [6]:
data

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,0,13926,NaN,150.0,0,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,0,2755,404.0,150.0,1,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,0,4663,490.0,150.0,2,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,0,18132,567.0,150.0,1,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,1,4497,514.0,150.0,1,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,3166995,0,3958217,39.00,0,1877,310.0,150.0,1,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179996,3166996,0,3958237,59.95,0,10075,514.0,150.0,1,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179997,3166997,0,3958241,34.00,0,6053,122.0,150.0,1,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179998,3166998,0,3958260,59.00,0,7726,555.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data_lb

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.000,0,12473,555.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.000,0,15651,417.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3287002,0,7415081,319.950,0,13844,583.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3287003,0,7415111,171.000,0,11556,309.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3287004,0,7415112,107.950,0,10985,555.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,3386996,0,10091528,368.990,0,13964,496.0,150.0,1,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,3386997,0,10091533,445.330,0,10616,583.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,3386998,0,10091544,15.226,2,9803,583.0,150.0,2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99999,3386999,0,10091549,34.742,2,16062,500.0,185.0,1,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def split_on_Xy(df):
    return df.drop(['isFraud'], axis=1), df['isFraud']

X_lb, y_lb = split_on_Xy(data_lb)
X_lb.shape, y_lb.shape

((100001, 393), (100001,))

In [9]:
data.sort_values(by=['TransactionID'], inplace=True)

X, y = split_on_Xy(data)
X.shape, y.shape

((180000, 393), (180000,))

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.6, shuffle=False)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((108000, 393), (72000, 393), (108000,), (72000,))

In [11]:
def get_auc_score(y_true, y_pred):
    return round(roc_auc_score(y_true, y_pred), 4)

In [12]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)

In [13]:
%%time
model = xgb.train(
    params={
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        
        'max_depth': 7,
        'learning_rate': 0.1,
        'num_parallel_tree': 1,
        'reg_lambda': 40,
        'gamma': 9
    },
    
    dtrain=dtrain,
    evals=[(dtrain, 'train'), (dvalid, 'valid')],
    verbose_eval=10,
    
    num_boost_round=1000,
    early_stopping_rounds=15,
    maximize=True
)

y_train_pred = model.predict(xgb.DMatrix(X_train))
y_valid_pred = model.predict(xgb.DMatrix(X_valid))

print(f'Overfitting: {round(get_auc_score(y_train, y_train_pred) - get_auc_score(y_valid, y_valid_pred), 2)}')

[0]	train-auc:0.67339	valid-auc:0.65586
[10]	train-auc:0.81212	valid-auc:0.81283
[20]	train-auc:0.86585	valid-auc:0.84822
[30]	train-auc:0.88235	valid-auc:0.85612
[40]	train-auc:0.89923	valid-auc:0.86663
[50]	train-auc:0.91006	valid-auc:0.86954
[60]	train-auc:0.91689	valid-auc:0.87362
[70]	train-auc:0.92342	valid-auc:0.87833
[80]	train-auc:0.92738	valid-auc:0.88020
[90]	train-auc:0.92987	valid-auc:0.88114
[100]	train-auc:0.93145	valid-auc:0.88251
[110]	train-auc:0.93173	valid-auc:0.88232
[114]	train-auc:0.93173	valid-auc:0.88232
Overfitting: 0.05
CPU times: user 5min 33s, sys: 2.77 s, total: 5min 35s
Wall time: 1min 49s


In [14]:
train_score = get_auc_score(y_train, model.predict(xgb.DMatrix(X_train)))
valid_score = get_auc_score(y_valid, model.predict(xgb.DMatrix(X_valid)))
test_score = get_auc_score(y_lb, model.predict(xgb.DMatrix(X_lb)))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.932, Valid-score: 0.882, Test-score: 0.868


Вывод: Небольшое переобучение на трейне в сравнении с валидационной выборкой и уже более существенное в сравнении с тестовой

### Задание 2: 

сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [18]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, train_size=0.6, shuffle=True)
X_valid.shape, X_test.shape, y_valid.shape, y_test.shape

((43200, 393), (28800, 393), (43200,), (28800,))

In [19]:
%%time
model = xgb.train(
    params={
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        
        'max_depth': 6,
        'learning_rate': 0.1,
        'num_parallel_tree': 1,
        'reg_lambda': 20,
        'gamma': 4
    },
    
    dtrain=dtrain,
    evals=[(dtrain, 'train'), (dvalid, 'valid')],
    verbose_eval=10,
    
    num_boost_round=1000,
    early_stopping_rounds=15,
    maximize=True
)

y_train_pred = model.predict(xgb.DMatrix(X_train))
y_valid_pred = model.predict(xgb.DMatrix(X_valid))

print(f'Overfitting: {round(get_auc_score(y_train, y_train_pred) - get_auc_score(y_valid, y_valid_pred), 2)}')

[0]	train-auc:0.67414	valid-auc:0.65170
[10]	train-auc:0.83136	valid-auc:0.82847
[20]	train-auc:0.86739	valid-auc:0.84700
[30]	train-auc:0.88491	valid-auc:0.85676
[40]	train-auc:0.90675	valid-auc:0.86634
[50]	train-auc:0.91806	valid-auc:0.87060
[60]	train-auc:0.92689	valid-auc:0.87560
[70]	train-auc:0.93254	valid-auc:0.87885
[80]	train-auc:0.93661	valid-auc:0.88128
[90]	train-auc:0.93989	valid-auc:0.88345
[100]	train-auc:0.94257	valid-auc:0.88477
[110]	train-auc:0.94501	valid-auc:0.88610
[120]	train-auc:0.94747	valid-auc:0.88630
[130]	train-auc:0.94907	valid-auc:0.88729
[140]	train-auc:0.95030	valid-auc:0.88749
[150]	train-auc:0.95236	valid-auc:0.88847
[160]	train-auc:0.95377	valid-auc:0.88878
[170]	train-auc:0.95517	valid-auc:0.88871
[174]	train-auc:0.95549	valid-auc:0.88849
Overfitting: 0.07
CPU times: user 7min 7s, sys: 2.44 s, total: 7min 10s
Wall time: 2min 11s


In [20]:
train_score = get_auc_score(y_train, model.predict(xgb.DMatrix(X_train)))
valid_score = get_auc_score(y_valid, model.predict(xgb.DMatrix(X_valid)))
test_score = get_auc_score(y_test, model.predict(xgb.DMatrix(X_test)))
lb_score = get_auc_score(y_lb, model.predict(xgb.DMatrix(X_lb)))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}, , Leaderbord-score: {round(lb_score, 3)}")

Train-score: 0.956, Valid-score: 0.89, Test-score: 0.887, , Leaderbord-score: 0.869


### Задание 3: 

Построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [21]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [22]:
%time

scores = create_bootstrap_metrics(y_train, model.predict(xgb.DMatrix(X_train)), roc_auc_score)

print(f'train: {calculate_confidence_interval(scores)}')

scores = create_bootstrap_metrics(y_test, model.predict(xgb.DMatrix(X_test)), roc_auc_score)

print(f'test: {calculate_confidence_interval(scores)}')

scores = create_bootstrap_metrics(y_valid, model.predict(xgb.DMatrix(X_valid)), roc_auc_score)

print(f'valid: {calculate_confidence_interval(scores)}')


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.7 µs
train: (0.9511798403405405, 0.959652180279372)
test: (0.8733682749339113, 0.9002242666232909)
valid: (0.8795322093188701, 0.8987531668554971)


Вывод: результат лидерборда лежит вне доверительных интервалов. Валидация не очень хорошая.

### Задание 4: 

выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [23]:
X_adv = pd.concat([X, X_lb], axis=0)
y_adv = np.hstack((
    np.zeros(len(X)), 
    np.ones(len(X_lb))
))

In [26]:
%%time
model_adv = xgb.train(
    params={
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc'
    },
    dtrain=xgb.DMatrix(X_adv, y_adv),
)

CPU times: user 31.4 s, sys: 1.51 s, total: 32.9 s
Wall time: 12.5 s


In [27]:
y_adv_pred = model_adv.predict(xgb.DMatrix(X_adv))
roc_auc_score(y_adv, y_adv_pred)

1.0

In [28]:
y_pred = model_adv.predict(xgb.DMatrix(X_train))

pd.cut(
    y_pred, bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]    108000
(0.1, 0.2]         0
(0.2, 0.3]         0
(0.3, 0.4]         0
(0.4, 0.5]         0
(0.5, 0.6]         0
(0.6, 0.7]         0
(0.7, 0.8]         0
(0.8, 0.9]         0
(0.9, 1.0]         0
dtype: int64

### Задание 5: 

сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [30]:
models, fold_train_score, fold_val_score = [], [], []

for train_idx, val_idx in KFold(n_splits=5).split(X, y):
    
    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    
    dtrain = xgb.DMatrix(X_train, y_train)
    dval = xgb.DMatrix(X_val, y_val)
    
    model = xgb.train(
        params={'booster': 'gbtree',
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'max_depth': 3,
                'learning_rate': 0.1,
                'num_parallel_tree': 1,
                'reg_lambda': 100,
                'gamma': 10},
        dtrain=dtrain,
        evals=[(dtrain, 'train'), (dval, 'valid')],
        verbose_eval=50,

        num_boost_round=1000,
        early_stopping_rounds=15,
        maximize=True
    )
    y_train_pred = model.predict(dtrain)
    y_val_pred = model.predict(dval)
    
    score_train = roc_auc_score(y_train, y_train_pred)
    score_val = roc_auc_score(y_val, y_val_pred)
    
    fold_train_score.append(score_train)
    fold_val_score.append(score_val)
    models.append(model)

mean_train_score = np.mean(fold_train_score)
mean_val_score = np.mean(fold_val_score)

print(f'train: {mean_train_score}', 
      f'val: {mean_val_score}',
      f'diff: {mean_train_score - mean_val_score}',
      sep='\n')

[0]	train-auc:0.61928	valid-auc:0.57191
[50]	train-auc:0.87771	valid-auc:0.85147
[100]	train-auc:0.89948	valid-auc:0.87222
[150]	train-auc:0.90809	valid-auc:0.87789
[200]	train-auc:0.91305	valid-auc:0.88248
[232]	train-auc:0.91463	valid-auc:0.88369
[0]	train-auc:0.63620	valid-auc:0.66988
[50]	train-auc:0.87380	valid-auc:0.86746
[100]	train-auc:0.89857	valid-auc:0.88510
[150]	train-auc:0.90698	valid-auc:0.89153
[200]	train-auc:0.91246	valid-auc:0.89477
[215]	train-auc:0.91246	valid-auc:0.89477
[0]	train-auc:0.61964	valid-auc:0.62534
[50]	train-auc:0.87397	valid-auc:0.87379
[100]	train-auc:0.89459	valid-auc:0.89029
[150]	train-auc:0.90376	valid-auc:0.89896
[200]	train-auc:0.90920	valid-auc:0.90323
[248]	train-auc:0.91176	valid-auc:0.90515
[0]	train-auc:0.60888	valid-auc:0.61320
[50]	train-auc:0.87284	valid-auc:0.86192
[100]	train-auc:0.89525	valid-auc:0.87690
[150]	train-auc:0.90437	valid-auc:0.88404
[200]	train-auc:0.90961	valid-auc:0.88753
[223]	train-auc:0.91054	valid-auc:0.88821
[0]	

In [31]:
mean_lb_score = []
for model in models:
    y_lb_pred = model.predict(xgb.DMatrix(X_lb))
    score = get_auc_score(y_lb, y_lb_pred)
    mean_lb_score.append(score)
    print('LB =', get_auc_score(y_lb, y_lb_pred))
    
print('mean LB =', round(np.mean(mean_lb_score), 4))

LB = 0.8614
LB = 0.863
LB = 0.8623
LB = 0.8611
LB = 0.8651
mean LB = 0.8626
